<a href="https://colab.research.google.com/github/MohammadRaziei/Deep-Learning-Course/blob/master/HW04/Q1/HW04_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW04 : Problem 1

Please check last updates and get the latest version from following link. it is more completed but it may have some updates after release it.

[<img src="https://img.icons8.com/clouds/50/000000/github.png"/>](https://github.com/MohammadRaziei/Deep-Learning-Course/blob/master/)  
github link : https://github.com/MohammadRaziei/Deep-Learning-Course/blob/master/HW04

In [1]:
!pip install music21
!pip install bs4

In [2]:
from keras.layers import Input, LSTM, Dense, TimeDistributed, Activation, BatchNormalization, Dropout, Bidirectional
from keras.models import Sequential
from keras.utils import Sequence
import keras.utils.np_utils as np_utils

import tensorflow as tf
import numpy as np


In [3]:
import os, shutil
def create_empty_folder(folder_name):
  try:
    try: shutil.rmtree(folder_name)
    except: pass
    os.mkdir(folder_name)
    print('"%s" is created.'%folder_name)
  except Exception as e: # work on python 3.x
    print('Some error! : '+ str(e))

def create_empty_folders(folders_list):
  for folder in folders_list:
    create_empty_folder(folder)

In [4]:
create_empty_folders(['SimpleRNN', 'LSTM'])

"guitar" is created.
"SimpleRNN" is created.
"LSTM" is created.


In [14]:
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import time
from tqdm.auto import trange


instrument = 'Guitar'

# Define URL components
url_base = 'https://www.mutopiaproject.org/cgibin/make-table.cgi?startat={}&searchingfor=&Composer=&Instrument={}&Style=&collection=&id=&solo=&recent=&timelength=&timeunit=&lilyversion=&preview='

# Define save directory.
save_dir = 'guitar'
create_empty_folder(save_dir)

# Set initial values
songNumber = 0
linkCount = 10

lim = 10 
cnt = 0

# Locate and download each MIDI file
for cnt in trange(lim):
    cnt += 1
    url = url_base.format(songNumber, instrument)
    html = urlopen(url)
    soup = BeautifulSoup(html.read())
    links = soup.find_all('a')
    linkCount = 0
    for link in links:
        href = link['href']
        if href.find('.mid') >= 0:
            linkCount = linkCount + 1
            urlretrieve(href, os.path.join(save_dir, "%d_%d.mid"%(cnt, linkCount)))
    if linkCount is 0: break
    songNumber += 10
    time.sleep(.1)

"guitar" is created.


In [16]:
from music21 import converter
import os

save_dir = 'guitar'

song_list = os.listdir(save_dir)


# test that stream is monotonic or not
def is_monophonic(stream):
    try:    length = len(instrument.partitionByInstrument(stream).parts)
    except: length = 0; 
    return length is 1

# Merge notes into chords
data_scores = [sng.chordify() for song in song_list for sng in (converter.parse(os.path.join(save_dir,song)),)]# if is_monophonic(sng)]
n
len(data_scores)

96

In [9]:
from music21 import note, chord
from tqdm.auto import tqdm


data_chords = [[] for _ in data_scores]
data_durations = [[] for _ in data_scores]
data_keys = []

# Extract notes, chords, durations, and keys
for i, song in enumerate(tqdm(data_scores)):
  data_keys.append(str(song.analyze('key')))
  for element in song:
      if isinstance(element, note.Note):
          data_chords[i].append(element.pitch)
          data_durations[i].append(element.duration.quarterLength)
      elif isinstance(element, chord.Chord):
          data_chords[i].append('.'.join(str(n) for n in element.pitches))
          data_durations[i].append(element.duration.quarterLength)

In [10]:
data_scores

[]

In [8]:
from sklearn.preprocessing import LabelEncoder

chord_encoder = LabelEncoder()
chord_encoder.fit(sorted(list({*[subsub for sub in data_chords for subsub in sub]})))

duration_encoder = LabelEncoder()
duration_encoder.fit(sorted(list({*[subsub for sub in data_durations for subsub in sub]})))

print('number of unique chords: ', len(chord_encoder.classes_))
print('10 instances of chords:', chord_encoder.classes_[0:10])
print()
print('number of unique duration: ', len(duration_encoder.classes_))
print('unique durations:', [str(s) for s in duration_encoder.classes_])

NameError: ignored

In [ ]:
from tqdm.auto import trange

sequenceLength = 32

# Define empty array for train data
train_chords = [];  train_durations = []
target_chords = []; target_durations = []

# Construct train and target sequences for chords and durations
for i in trange(len(data_chords)):
    chord_list = chord_encoder.transform(data_chords[i])
    duration_list = duration_encoder.transform(data_durations[i])
    for t in range(len(chord_list) - sequenceLength):
        train_chords.append(chord_list[t:t+sequenceLength])
        target_chords.append(chord_list[t+1])

        train_durations.append(duration_list[t:t+sequenceLength])
        target_durations.append(duration_list[t+1])

train_chords = np.array(train_chords);    train_durations = np.array(train_durations)
target_chords = np.array(target_chords);  target_durations = np.array(target_durations)

In [ ]:
dir(chords_encoder)